In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp
import itertools
import matplotlib.pyplot as plt

# from utils_opt import *

%load_ext autoreload
%autoreload 2

# Test active learning approach

In [3]:
# make fake data
n, d = 100, 5
Sigma = np.random.normal(size=(d, d))
Sigma = Sigma.T @ Sigma
mu = np.random.normal(size=d)
# assume moving average system
X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n)

beta = np.random.normal(size=d)
f = X@beta + np.random.normal(size=n)

In [4]:
k = 7
Xk = X[:k]

In [5]:
# covariance matrix
K = np.array([np.diagonal((Xk[i] - Xk) @ (Xk[i] - Xk).T) for i in range(k)])
K = np.exp(-0.5*K)
K_inv = np.linalg.inv(K)

In [6]:
# b matrix
normsqs = 0.5*np.diagonal(Xk @ Xk.T)
B = np.array([np.log(K[i]) + normsqs[i] + normsqs for i in range(k)])

# m tensor
M = np.array([Xk[i] + Xk for i in range(k)])

In [7]:
x = cp.Variable(d)
y = cp.hstack([cp.exp(-(1/2)*((x - Xk[i]).T @ (x - Xk[i]))) for i in range(k)])
obj = cp.log(cp.quad_form(y, K_inv))

In [12]:
cp.vstack([cp.exp(-1*(cp.quad_form(x, np.identity(d)) + x.T @ M[i,j] - B[i,j])) for i in range(k) for j in range(k)])

Expression(UNKNOWN, NONNEGATIVE, (49, 1))

In [265]:
obj = cp.log(cp.sum(cp.exp(-cp.vstack([cp.quad_form(x, np.identity(d)) + x.T @ M[i,j] - B[i,j] for i in range(k) for j in range(k)]))))

In [266]:
# problem
prob = cp.Problem(cp.Maximize(obj))
prob.solve(solver='SCS')

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
exp(-Vstack(QuadForm(var23113, [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]) + var23113 @ [ 1.58130646 -0.7423973  -3.98291009 -6.6476665   6.0118778 ] + -24.81235029671911, QuadForm(var23113, [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]) + var23113 @ [ 2.31031818e+00  6.39634169e-03 -3.25687260e+00 -7.05627328e+00
  4.67411933e+00] + -24.57768179958472, QuadForm(var23113, [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]) + var23113 @ [-0.80731051 -1.34159983 -6.72087419 -5.06391457  3.83343481] + -37.60121047664949, QuadForm(var23113, [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]) + var23113 @ [ 1.85402294  0.53096919 -6.11836484 -6.31170487  4.38552341] + -31.816464654127717, QuadForm(va

In [212]:
# objective function
quad_array = [-cp.quad_form(x, np.identity(d)) + x.T @ M[i,j] - B[i,j] for i in range(k) for j in range(k)]
quad_array = cp.vstack(quad_array)

# obj = cp.log(1 - cp.quad_form(x, np.identity(d)) * 
# obj = cp.exp(cp.quad_form(x, np.identity(d)))*cp.sum(cp.exp(quad_array))
obj = cp.log_sum_exp(quad_array)

In [144]:
quad_array

Expression(CONVEX, UNKNOWN, (1, 49))

In [94]:
x.T @ M

ValueError: Expressions of dimension greater than 2 are not supported.